# AI and Blockchain: Fall 2023
## Lab 02 : Secure and Private AI 

<p><b>Instructions:</b> Complete the code in the designated cells, and upload the completed python notebook on LMS prefixed with your RCS ID, i.e., "senevo_lab02.ipynb". </p>

<p>Total points: <b>100</b></p>

<p>Assigned: <b>Oct 6, 2023</b></p>
<p>Due: <b>9:59 AM Oct 20, 2023</b></p>

<hr>

### Task 0: Please Type Your Information

RCS ID: 

Name: 

Level (4000/6000): 

## Differential Privacy with a Toy Dataset

Step one is to create our dataset, i.e., "database" - we will do this by initializing a random list of 1s and 0s (the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [ ]:
import torch

num_entries = 5000
db = torch.rand(5000) > 0.5
db.int()[:10]

In [ ]:
db.shape

Key to the definition of differential privacy is the ability to ask the question, "When querying a database if I removed someone from the database, would the output of the query be any different?". To check this, we must construct what we term "parallel databases," which are simply databases with one entry removed.

### Task 1: Create parallel databases (2 points)

Create a list of every parallel database to the one currently in the "db" variable. The code to create a single parallel database is provided for you.

In [ ]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))
                      
def get_parallel_dbs(db):
    parallel_dbs = list()
    
    # Type your code here
    
    return parallel_dbs

Now, let's create the database and the parallels.

In [ ]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

## Evaluating The Differential Privacy of a Function

Let's make our first "database query" a simple sum by counting the number of 1s in the database.

In [ ]:
db, pdbs = create_db_and_parallels(5000)

In [ ]:
def query(db):
    return db.sum()

In [ ]:
full_db_result = query(db)
full_db_result

In [ ]:
pdb0_result = query(pdbs[0])
pdb0_result

In [ ]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [ ]:
sensitivity

## Evaluating the Privacy of a Function

We can measure the difference between each parallel db's query result and the query result for the entire database by calculating the max value (1). This value is called "sensitivity," corresponding to the function we chose for the query. The "sum" query will always have a sensitivity of exactly 1. We can also calculate sensitivity for other functions. The steps for evaluating the sensitivity include the following:

* Initialize a database of the correct size
* Initialize all parallel databases
* Run the query over all databases
* Correctly calculate sensitivity
* Return the sensitivity

In [ ]:
def sensitivity(query, n_entries=1000):

    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
            
    return max_distance

### Task 2: Calculate sensitivity for the "mean" function (3 points)

In [ ]:
# Type your code here. You should print the sensitivity for mean

## A Basic Differencing Attack

Let's construct a database and then demonstrate how you can use two different sum queries to expose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [ ]:
db, _ = create_db_and_parallels(100)

In [ ]:
pdb = get_parallel_db(db, remove_index=10)

In [ ]:
db[10].int()

In [ ]:
sum(db)

In [ ]:
# differencing attack using sum query
sum(db) - sum(pdb)

### Task 3: Demonstrate the differencing attack using the mean query (5 points)

In [ ]:
# Type your code here

## Local Differential Privacy
### Randomized Response

Let's say we have a group of people we wish to survey about a taboo behavior (e.g., nose picking, drug addiction, criminal behavior, cheating on their spouse) that we think they will lie about. 
We are not law enforcement; we are just trying to collect statistics to understand the higher-level trend in society using AI. 
So, how do we do this?

One technique is to add randomness to each person's response by giving each person the following instructions (assuming we are asking a simple yes/no question):

* Flip a coin two times.
* If the first coin flip is heads, answer honestly
* If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Each person is now protected with "plausible deniability." If they answer "Yes" to the question "Have you committed X crime?", it might be because they actually did or because they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is centered around 70% because 70% averaged with 50% (a coin flip) is 60%, which is the result we obtained.

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability), the less accurate the results.

Let's implement this local DP for our database before!

In [ ]:
def query(db):

    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) < 0.5).float()
    second_coin_flip = (torch.rand(len(db)) < 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    private_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return private_result, true_result

### Task 4: Show how the private result (with noise) and the true result (without noise) change for databases of size 10, 100, 1000, and 10000. (5 points)
*Hint: Chart the results in a histogram for easy comparison.*

In [ ]:
# Type your code here


## Varying Amounts of Noise

Let's augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, let's bias the coin flip to be higher or lower and then run the same experiment.

In [ ]:
def query(db, noise=0.2):
    
    true_result = torch.mean(db.float())

    first_coin_flip = (torch.rand(len(db)) < noise).float()
    second_coin_flip = (torch.rand(len(db)) < 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    sk_result = augmented_database.float().mean()

    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

    return private_result, true_result

### Task 5: Show how the private result (with noise) and the true result (without noise) changes for databases of size 10, 100, 1000, and 10000 with varying noise levels (noise=[0.1, 0.2, ..., 0.9]). (5 points)
*Hint: A convenience function for plotting results is provided for you.*

In [ ]:
import matplotlib.pyplot as plt

def plot_results(db_size, noises, private_results, true_results):
    # plots the results on varying noise levels on a specific database size

    plt.scatter(noises, private_results, color = "blue", label = "With Noise")
    plt.scatter(noises, true_results, color = "red", label = "Without Noise")
    
    plt.xlabel("Noise")
    plt.ylabel("Results")
    plt.title("DB Size : " + str(db_size))

    plt.legend()
    
    plt.show()

In [ ]:
# Type your code here

## The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differential Privacy" because we added noise to each data point individually. This is necessary for some situations wherein the data is so sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy.

However, alternatively, we can add noise after a function has aggregated data. This kind of noise can allow for similar levels of protection with a lower effect on accuracy. However, participants must be able to trust that no one looked at their data points before the aggregation took place. In some situations, this works out well, but it is less realistic in others (such as an individual hand-surveying a group of people).

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [ ]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

The idea here is that we want to add noise to the output of our function. We can add two different kinds of noise - Laplacian Noise and Gaussian Noise. However, before we do so, we need to dive into the formal definition of Differential Privacy.

### Epsilon
Let's unpack the intuition of this for a moment.

**Epsilon Zero**: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputs the same value as the full database. 

**Epsilon One**: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

### Delta

Delta is the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than others. 

## How To Add Noise for Global Differential Privacy

Let's learn how to take a query and add varying amounts of noise to satisfy a certain degree of differential privacy. In particular, we will leave behind the previously discussed Local Differential privacy and instead focus on Global differential privacy.

To sum up, this is about adding noise to our query's output to satisfy a certain epsilon-delta differential privacy threshold.

We can add two kinds of noise - Gaussian Noise and Laplacian Noise. Generally speaking, Laplacian is better, but both are still valid. 

How much noise should we add?
The amount of noise necessary to add to the output of a query is a function of four things:
1. the type of noise (Gaussian/Laplacian)
2. the sensitivity of the query/function
3. the desired epsilon (ε)
4. the desired delta (δ)

Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, we know we will have a privacy leakage <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. However, we can tune the parameters such that we have very low epsilon where the delta is non-zero, but we'll ignore them for now.

If we query the database multiple times - we can add the epsilons (even if we change the amount of noise and their epsilons are not the same).

## Create a Differentially Private Query

Let's create a query function that sums over the database and adds just the right amount of noise to satisfy an epsilon constraint. 

In [ ]:
import numpy as np

epsilon = 0.0001

db, pdbs = create_db_and_parallels(100)

def sum_query(db):
    return db.sum()

def mean_query(db):
    return torch.mean(db.float())

def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(db) + noise



In [ ]:
laplacian_mechanism(db, sum_query, 1)

In [ ]:
laplacian_mechanism(db, mean_query, 1/100)

# Differential Privacy for Deep Learning

## An Example Scenario: A Health Neural Network

Let's consider a scenario - you work for a hospital and have a large collection of images of your patients. However, you do not know what's in them. You would like to use these images to develop a neural network to classify them automatically. However, since your images aren't labeled, they aren't sufficient to train a classifier.

However, you realize you can reach out to 10 partner hospitals with annotated data. You plan to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding patient information. Thus, you will use the following technique to train a classifier that protects patients' privacy in the other hospitals.

1. You ask each of the 10 hospitals to train a model on their datasets (all of which have the same labels).
2. Use each of the 10 partner models on your local dataset, generating 10 labels for each of your data points.
3. For each local data point (now with 10 labels), you perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. You need to add laplacian noise to make this "Differentially Private" to a certain epsilon/delta constraint.
4. Finally, you retrain a new model on your local dataset, which now has labels. This trained model will be our final "Differentially Private" model.

Assuming you're already familiar with how to train/predict a deep neural network, we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models," which were trained directly on private data). Each label is chosen from 10 possible labels (categories) for each image.

In [ ]:
import numpy as np

num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [ ]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions
preds

Prediction for the first data item from all the teachers.

In [ ]:
preds[0]

All the predictions from the first teacher.

In [ ]:
preds[:,0]

### Task 6: Combine these label predictions in a differentially private way. You may use an epsilon value of 0.1. (10 Points)

In [ ]:
# Type your code here

# Federated Learning

Federated Learning is a machine learning technique that allows multiple clients to collaborate on a model training task without sharing their data with a central server. Instead, the clients train their models locally and exchange model updates with each other or with a central aggregator to improve the global model.

### Learning Objectives

By completing this part of the lab, you will learn:

* How to use PyTorch to implement a federated learning algorithm
* How to distribute the training process across multiple clients
* How to aggregate model weights from multiple clients to improve the global model performance
* How to evaluate the performance of a federated learning algorithm


### Set up the environment

We start by installing the required packages and importing the necessary modules.
It is highly recommended that you check out the [FedLab GitHub](https://github.com/SMILELab-FL/FedLab) repo and use the code directly from there.
Alternatively, using the following command, you can install fedlab and torchvision.

Reference: Tutorial for FedLab users (https://github.com/SMILELab-FL/FedLab/blob/master/tutorials/pipeline_tutorial.ipynb)

In [ ]:
!pip install fedlab

If you clone the FedLab GitHub repo, here's an example code snippet on how you may configure your environment.

In [ ]:
import sys
# sys.path.append("../")

# configuration
from munch import Munch
from fedlab.models.mlp import MLP

from fedlab.utils.functional import evaluate
from fedlab.core.standalone import StandalonePipeline

from torch import nn
from torch.utils.data import DataLoader
import torch
import torchvision

import matplotlib.pylab as plt


### Task 7: Load the MNIST dataset [3 points]

You will be using the MNIST dataset for this lab. You may load the dataset using the TorchVision API or use Fedlab's version of the dataset.
FedLab provides the necessary module for users to partition their datasets. Additionally, various implementations of datasets partition for federated learning are available at the [URL](https://github.com/SMILELab-FL/FedLab/tree/master/datasets).

In [ ]:
!mkdir ./datasets

In [ ]:
# Type your code here.

Once you load the dataset, you may use the following function to visualize the digits.

In [ ]:
IMAGE_SIZE = 28

def show_data(data_sample):
    """plot out data samples as images"""
    plt.imshow(data_sample[0].numpy().reshape(
        IMAGE_SIZE, IMAGE_SIZE), cmap='gray')
    plt.title('y = ' + str(data_sample[1]))


### Task 8: Visualize a single digit [2 points]

Using the above function, visualize any digit in the dataset.

In [ ]:
# Type your code here.

### Task 9: Define the model architecture [5 points]
You should use PyTorch and the FedLab APIs to define your model architecture. 

*Hint: You can use a simple neural network or the `fedlab.models.mlp`.*

In [ ]:
# Type your code here


### Task 10: Implement the client-side training logic [5 points]

Implement the client-side training logic, which should involve loading the local data, training the model on the local data, and sending the updated weights to the server. 

*Hint: For this task, you may use the `fedlab.contrib.algorithm.basic_client`.*

In [ ]:
# Type your code here


### Task 11: Implement the server-side aggregation logic [5 points]

Implement the server-side aggregation logic, which should involve receiving the updated weights from each client, aggregating the weights, and sending the updated global model weights back to each client. 

*Hint: You may use the `fedlab.contrib.algorithm.basic_server`.*

In [ ]:
# Type your code here

### Task 12: Train the federated learning algorithm [5 points]

Once you have implemented the client-side training and server-side aggregation logic, you should train the model. 

*Hint: We recommend creating a subclass of the `fedlab.core.standalone.StandalonePipeline` for this purpose.*

In [ ]:
# Type your code here.

### Task 13: Evaluate the federated learning algorithm [5 points]

Now, evaluate the performance of the federated learning algorithm. You can do this by computing the accuracy of the global model on the test set. You can also compare the performance of the federated learning algorithm with that of a centralized learning algorithm, where the data is collected and trained on a single server.

Use the convenience functions below for your evaluation.

In [ ]:
import matplotlib.pyplot as plt


def generate_loss_accuracy_charts(label, loss_list,  accuracy_list):

    fig, ax1 = plt.subplots()

    # create a subplot for the current key
    # ax = fig.add_subplot(len(data1), 1, i+1)

    color = 'tab:green'
    ax1.set_xlabel('epoch', color=color)

    color = 'tab:red'
    ax1.plot(loss_list, color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis='y', color=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)
    ax2.plot(accuracy_list, color=color)
    ax2.tick_params(axis='y', color=color)

    ax1.set_title(label)

    fig.tight_layout()

    # show the figure
    plt.show()


In your analysis, you could analyze misclassified images using the following function.

In [ ]:
def print_first_five_misclassified_samples(dataset, model):
    Softmax_fn = nn.Softmax(dim=-1)
    count = 0
    for x, y in dataset:
        z = model(x.unsqueeze(0))
        _, yhat = torch.max(z, 1)
        if yhat != y:
            show_data((x, y))
            plt.show()
            print("yhat:", yhat)
            print("probability of class ", torch.max(Softmax_fn(z)).item())
            count += 1
        if count >= 5:
            break


In [ ]:
# Type your code here.


### Task 14: Fine-tune the hyperparameters [5 points]

You should experiment with different hyperparameters to improve the performance of the federated learning algorithm. 

*Hint: You can try adjusting the num clients, learning rate, the batch size, or the number of communication rounds.*



In [ ]:
# Type your code here.

### Task 15: A description of the federated learning algorithm you implemented. [5 points]


Type your answer here (this is a written answer, use markdown syntax as necessary).

### Task 16: A summary of the performance of the federated learning algorithm [5 points]

Type your answer here (this is a written answer, use markdown syntax as necessary).

### Task 17: A comparison of the performance of the federated learning algorithm with the performance of a centralized learning algorithm [5 points]

*Hint: To implement centralized training you set num client to be 1 and try out a few different learning rates.*

*Hint: You must also provide a written answer in markdown syntax analyzing the results you obtained.*

In [ ]:
# Type your code here 

Type your answer here (this is a written answer, use markdown syntax as necessary).


### Task 18: A discussion of the hyperparameters you fine-tuned and their impact on the performance of the federated learning algorithm [5 points]

In [ ]:
# Type any supporting code here

Type your answer here (this is a written answer, use markdown syntax as necessary).

### Task 19: Implement the aggregator using a Solidity smart contract. [10 points]
You must write Solidity code to get full points for this question. The contract should be syntactically correct (i.e., compile).

Some Hints: 
* Create structs for the Participants (i.e., clients) and Global Model (i.e., the server). 
* Some useful functions to include in the smart contract: addParticipant, updateParticipantParameters, aggregate
* The `aggregate` function should aggregate the model parameters from all the participants, and you are welcome to utilize any relevant algorithm for this purpose.

Since the code is in Solidity, place it inside the markdown cell in the "javascript" environment.

```javascript
//Type your code here. 
```


### Task 20: Deploying and using the aggregator smart contract. [5 points]
Please describe the steps you would take to deploy the smart contract you developed above on an Ethereum test network (you need not deploy the contract). How would the participants/clients interact with the smart contract?

Type your answer here (this is a written answer, use markdown syntax as necessary).